In [ ]:
import numpy as np
from typing import Tuple, Collection, Any, Dict, List, Optional, Set

In [ ]:
WorldWall = Tuple[Tuple[float, float], Tuple[float,float]]
WorldWalls = Set[WorldWall]

In [ ]:
def RLE_1D(grid: np.ndarray) -> List[List[int]]:
    res: List[List[int]] = list()
    for major in grid:
        run: int = 1
        last: int = major[0]
        subres: List[int] = list()
        for minor in major[1:]:
            if minor == last:
                run += 1
            else:
                subres.append(run)
                run = 1
                last = minor
        #intentially don't add last run - no new wall
        res.append(subres)
    return res 

In [ ]:
def RLE_2D(grid: np.ndarray) -> WorldWalls:

    class WallLines(Dict[float, List[Tuple[float, float]]]):

        _inverted: bool

        def __init__(self, inverted: bool = False, *args, **kwargs):
            super().__init__(*args, **kwargs)
            self._inverted = inverted

        def add(self, major: float, minor: float):
            if major not in self:
                self[major] = [(minor, minor+1)]
                return
            
            last = self[major][-1]

            if minor == last[1]:
                self[major][-1] = (last[0], minor+1)
            else:
                self[major].append((minor, minor+1))

        @property
        def lines(self) -> WorldWalls:
            if self._inverted:
                return set([((start, major), (end, major)) for major, segment in self.items()  for start, end in segment])
            
            else:
                return set([((major, start), (major, end)) for major, segment in self.items()  for start, end in segment])

    walls_x = WallLines()
    walls_y = WallLines(inverted=True)

    for y, rles in enumerate(RLE_1D(grid)):
        distance: int = 0
        for run in rles:
            distance += run
            walls_x.add(distance, y)

    for x, rles in enumerate(RLE_1D(grid.T)):
        distance: int = 0
        for run in rles:
            distance += run
            walls_y.add(distance, x)

    return set().union(walls_x.lines, walls_y.lines)

In [ ]:
import cv2

base_img = cv2.imread("map4.png", cv2.IMREAD_GRAYSCALE)
base = (base_img > base_img.mean(axis=-1)).astype(int)


In [ ]:
import matplotlib.pyplot as plt


print(base[::-1,:])

walls = RLE_2D(base)

print(walls)

ax = plt.subplot()
ax.invert_yaxis()

ax.set_xlim(0, base.shape[1])
ax.set_ylim(0, base.shape[0])

for endpoints in walls:
    ax.plot(*zip(*endpoints))

plt.show()